Adding Wind Pressure Coefficients to Surfaces in an E+ IDF File using Python
==================================================================================================

Introduction
------------

In this tutorial, we will walk you through a Python script designed to add wind pressure coefficients to surfaces in an EnergyPlus IDF file. This can be particularly useful when you're modelling airflow and want to understand how the building's geometry interacts with the wind.

Prerequisites
-------------

Before we begin, ensure you have the following:

1.  Basic knowledge of Python.
2.  EnergyPlus and the `Energy+.idd` file.
3.  Python's EPPY library installed. If not, you can install it using pip: `pip install eppy`.


The `Energy+.idd` file, often simply referred to as the IDD, is the Input Data Dictionary for EnergyPlus. This file defines all the objects and fields that an IDF (Input Data File) will have. The IDD is crucial for software and scripts (like the one using EPPY in your previous query) to understand the structure of an IDF.

Here's how you can find the `Energy+.idd` file:

1.  Installation Directory: When you install EnergyPlus, the IDD is usually placed in the main installation directory of EnergyPlus.

    For example, on a Windows system, if you've installed EnergyPlus 9.3, the path might be something like:

    makefileCopy code

    `C:\EnergyPlusV9-3-0\EnergyPlusIDD.idd`

    On Linux, it might be under `/usr/local/EnergyPlus-9-3-0/` or a similar location, depending on how you've installed it.

    On macOS, if you've installed EnergyPlus using the installer package, the path might be:

    Copy code

    `/Applications/EnergyPlus-9-3-0/`

2.  EnergyPlus Toolkits: Some toolkits or GUI-based applications that work with EnergyPlus might keep their own copy of the IDD. If you're using such a toolkit, you might find the IDD in its directories.

3.  Search Your System: If you're unsure where EnergyPlus was installed, you can use system search functions:

    -   On Windows, you can use the search bar in the File Explorer.
    -   On Linux, you can use commands like `find` or `locate` (if `updatedb` has been run previously).
    -   On macOS, you can use Spotlight or Finder.
4.  Check Documentation: The EnergyPlus documentation might provide information on the installation directory, which can guide you to the location of the IDD.

When referencing the IDD in scripts or software, it's good practice to always point to the version of the IDD that matches the version of your EnergyPlus IDF files. This ensures compatibility and that all objects and fields are recognized correctly.



The Script
----------

The script you'll be working with uses the EPPY library to interact with EnergyPlus IDF files. We'll start by importing necessary modules, setting paths, and then diving into the core logic of the script.

In [1]:
# Importing necessary libraries
from eppy import modeleditor
from eppy.modeleditor import IDF

# Define paths
IDF_PATH = 'data/in.idf'  # This is your EnergyPlus IDF file.
IDD_PATH = 'idd/Energy+_23_1_0.idd'    # This file defines all the objects and fields that an IDF will have. It is distributed with EnergyPlus.

# Load the IDF file using EPPY
IDF.setiddname(IDD_PATH)    # Setting the IDD file so EPPY knows how to read the IDF file.
idf1 = IDF(IDF_PATH)


In [2]:
REF_IDF_PATH = 'data/box22_reference.idf'  # This is your EnergyPlus IDF file.
idf_ref = IDF(REF_IDF_PATH)
Airflownetwork_Multizone_Component_Detailedopening = idf_ref.idfobjects['AirflowNetwork:MultiZone:Component:DetailedOpening'][0]

Note: Ensure both `box_clean.idf` and `Energy+.idd` are in the current working directory or provide the full path to the files.

### Wind Pressure Coefficient Array

The first step is to define the wind direction and its corresponding coefficient. This is crucial to understand how wind affects different surfaces at various angles.

In [3]:
import pandas as pd
cp_data = pd.read_csv('data/Cp.csv')
cp_data.head()
# Fix cp csv
coords_df = cp_data[cp_data['angle'] == 0][['u_ref', 'x', 'y', 'z']]
coords_df
cp_list = []
for angles in cp_data['angle'].unique():
    cp_list.append(cp_data[cp_data['angle'] == angles]['c_p'].tolist())

# add the cp_list to the coords_df with the column name 'c_p_<angle>'
for i, angles in enumerate(cp_data['angle'].unique()):
    coords_df['c_p_{}'.format(angles)] = cp_list[i]


xyz_df = coords_df[['x', 'y', 'z','c_p_0']]
sampled_df = coords_df[['x', 'y', 'z','c_p_0']].copy()

# Define translation values
x_trans = 851.9646-101.854
y_trans = 883.6051+16.43799
z_trans = 69.24877-72.39

# Fine tuning
point_A = (920.479730342372, 936.878514625527, 58.3200021362305)
point_B = (920.59058, 936.92212, 58.17876999999999)

# Add translate values to the sampled_df
sampled_df['x'] += x_trans + point_B[0] - point_A[0]
sampled_df['y'] += y_trans + point_B[1] - point_A[1]
sampled_df['z'] += z_trans + point_B[2] - point_A[2]


# Assuming x_trans, y_trans, z_trans, point_A, and point_B are already defined
coords_df['x'] += x_trans + point_B[0] - point_A[0]
coords_df['y'] += y_trans + point_B[1] - point_A[1]
coords_df['z'] += z_trans + point_B[2] - point_A[2]

# Convert DataFrame to a NumPy array
coords_array = coords_df[['x', 'y', 'z']].to_numpy()
import numpy as np
from scipy.spatial import cKDTree
# Build a KDTree for faster nearest-neighbor search
kdtree = cKDTree(coords_array)

In [4]:


def get_cp(surface, kdtree=kdtree, coords_array=coords_array, threshold=None):

    # Extracting all vertices from the surface
    vertices = [(getattr(surface, f"Vertex_{i}_Xcoordinate"),
                 getattr(surface, f"Vertex_{i}_Ycoordinate"),
                 getattr(surface, f"Vertex_{i}_Zcoordinate")) 
                for i in range(1, int(len([field for field in surface.fieldnames if 'Vertex' in field]) / 3) + 1)]

    # Find closest points in the KDTree
    distances, indices = kdtree.query(vertices, k=1)

    # Check if any distance exceeds the threshold
    if threshold and np.any(distances > threshold):
        return [-9999] * 8

    # Use the indices to fetch the required rows from the original DataFrame
    closest_rows = coords_df.iloc[indices]
    cp_columns = ['c_p_0', 'c_p_45', 'c_p_90', 'c_p_135', 'c_p_180', 'c_p_225', 'c_p_270', 'c_p_315']
    avg_cp = closest_rows[cp_columns].mean().tolist()

    return avg_cp

In [5]:
wind_pressure_coefficient_array = idf1.newidfobject("AirflowNetwork:MultiZone:WindPressureCoefficientArray",
                               Name="Cp Data",
                               Wind_Direction_1 = 0,
                               Wind_Direction_2 = 45,
                               Wind_Direction_3 = 90,
                               Wind_Direction_4 = 135,
                               Wind_Direction_5 = 180,
                               Wind_Direction_6 = 225,
                               Wind_Direction_7 = 270,
                               Wind_Direction_8 = 315)

Here, we're setting up a new `AirflowNetwork:MultiZone:WindPressureCoefficientArray` object which describes how the wind pressure coefficient (Cp) varies with wind direction.

### Sample cp values

These are sample Cp values for each wind direction. For this tutorial, we are using a predefined set. In a real-world scenario, these values might be derived from wind tunnel testing or CFD simulations.

In [6]:
# Sample cp values
#CP_values = [-0.6,-0.5,-0.4,-0.5,-0.6,-0.5,-0.4,-0.5]
building_objects = idf1.idfobjects['BUILDINGSURFACE:DETAILED']
fenestration_objects = idf1.idfobjects['FENESTRATIONSURFACE:DETAILED']


In [7]:
import plotly.graph_objects as go
# Create an empty np array with rows = len(fenestration_objects) and columns = 3
import numpy as np
cp_array = np.empty((len(fenestration_objects), 3))
points = []
for surface in fenestration_objects:
    coordinates = [
            (surface.Vertex_1_Xcoordinate, surface.Vertex_1_Ycoordinate, surface.Vertex_1_Zcoordinate),
            (surface.Vertex_2_Xcoordinate, surface.Vertex_2_Ycoordinate, surface.Vertex_2_Zcoordinate),
            (surface.Vertex_3_Xcoordinate, surface.Vertex_3_Ycoordinate, surface.Vertex_3_Zcoordinate),
            (surface.Vertex_4_Xcoordinate, surface.Vertex_4_Ycoordinate, surface.Vertex_4_Zcoordinate)
        ]
    #extend the points list with the coordinates
    points.extend(coordinates)
# Convert the points list to a NP array
points_array = np.array(points)


# Scatter3D from sampled_df
trace1 = go.Scatter3d(
    x=sampled_df['x'],
    y=sampled_df['y'],
    z=sampled_df['z'],
    mode='markers',
    marker=dict(size=2, color='red') # red color
)

# Scatter3D from points_array
trace2 = go.Scatter3d(
    x=points_array[:,0],
    y=points_array[:,1],
    z=points_array[:,2],
    mode='markers',
    marker=dict(size=2, color='blue') # blue color
)

# Combine traces and set the layout
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z', range=[-50,250]),
    )
)

fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.show()



### Adding Cp values to Building Surfaces

Now, we'll loop through each building surface in the IDF. For every surface exposed to the outdoors, we'll assign the Cp values:

In [8]:
len(fenestration_objects)

2611

In [9]:

node_height = 6.5 # Placeholder
opening_array = []
null_cp_counter = 0
for i, surface in enumerate(fenestration_objects):
    if surface.View_Factor_to_Ground!=0:
        #print(f'Adding wind pressure coefficients for {surface.Name}')
        
        CP_values = get_cp(surface, threshold=1)
        
        # Count the number of times the CP_values is -9999
        if CP_values == [-9999] * 8:
            null_cp_counter += 1

        print(f'Number of surfaces processed: {i+1}/{len(fenestration_objects)}, Number of null CP values: {null_cp_counter}', end='\r')
        #print(f'Adding wind pressure coefficients for {surface.Name}')
        idf1.newidfobject("AirflowNetwork:MultiZone:WindPressureCoefficientValues",
                          Name=surface.Name,
                          AirflowNetworkMultiZoneWindPressureCoefficientArray_Name = wind_pressure_coefficient_array.Name,
                          Wind_Pressure_Coefficient_Value_1 = CP_values[0],
                          Wind_Pressure_Coefficient_Value_2 = CP_values[1],
                          Wind_Pressure_Coefficient_Value_3 = CP_values[2],
                          Wind_Pressure_Coefficient_Value_4 = CP_values[3],
                          Wind_Pressure_Coefficient_Value_5 = CP_values[4],
                          Wind_Pressure_Coefficient_Value_6 = CP_values[5],
                          Wind_Pressure_Coefficient_Value_7 = CP_values[6],
                          Wind_Pressure_Coefficient_Value_8 = CP_values[7])
        #print(f'Adding external node for {surface.Name}')
        idf1.newidfobject("AirflowNetwork:MultiZone:ExternalNode",
                            Name=surface.Name + '.EN', # Don't know why this is .EN
                            External_Node_Height = node_height,
                            Wind_Pressure_Coefficient_Curve_Name = surface.Name,
                            Symmetric_Wind_Pressure_Coefficient_Curve = 'No',
                            Wind_Angle_Type = 'Absolute')
        #print(f'Adding detailed opening settings for {surface.Name}')
        new_detailed_opening = idf1.newidfobject("AirflowNetwork:MultiZone:Component:DetailedOpening")
        
        # Set properties for the DetailedOpening object:
        new_detailed_opening.Name = surface.Name
        new_detailed_opening.Air_Mass_Flow_Coefficient_When_Opening_is_Closed = 0.00014
        new_detailed_opening.Air_Mass_Flow_Exponent_When_Opening_is_Closed = 0.65
        new_detailed_opening.Type_of_Rectangular_Large_Vertical_Opening_LVO = "NonPivoted"
        new_detailed_opening.Extra_Crack_Length_or_Height_of_Pivoting_Axis = 0
        new_detailed_opening.Number_of_Sets_of_Opening_Factor_Data = 2

        # Opening Factor 1 properties
        new_detailed_opening.Opening_Factor_1 = 0
        new_detailed_opening.Discharge_Coefficient_for_Opening_Factor_1 = 0.65
        new_detailed_opening.Width_Factor_for_Opening_Factor_1 = 0
        new_detailed_opening.Height_Factor_for_Opening_Factor_1 = 0
        new_detailed_opening.Start_Height_Factor_for_Opening_Factor_1 = 0

        # Opening Factor 2 properties
        new_detailed_opening.Opening_Factor_2 = 1
        new_detailed_opening.Discharge_Coefficient_for_Opening_Factor_2 = 0.65
        new_detailed_opening.Width_Factor_for_Opening_Factor_2 = 1e-08
        new_detailed_opening.Height_Factor_for_Opening_Factor_2 = 1
        new_detailed_opening.Start_Height_Factor_for_Opening_Factor_2 = 0

        # Opening Factor 3 properties
        new_detailed_opening.Opening_Factor_3 = 0
        new_detailed_opening.Discharge_Coefficient_for_Opening_Factor_3 = 0
        new_detailed_opening.Width_Factor_for_Opening_Factor_3 = 0
        new_detailed_opening.Height_Factor_for_Opening_Factor_3 = 0
        new_detailed_opening.Start_Height_Factor_for_Opening_Factor_3 = 0

        # Opening Factor 4 properties
        new_detailed_opening.Opening_Factor_4 = 0
        new_detailed_opening.Discharge_Coefficient_for_Opening_Factor_4 = 0
        new_detailed_opening.Width_Factor_for_Opening_Factor_4 = 0
        new_detailed_opening.Height_Factor_for_Opening_Factor_4 = 0
        new_detailed_opening.Start_Height_Factor_for_Opening_Factor_4 = 0

        opening_array.append(new_detailed_opening)

idf1.idfobjects["AirflowNetwork:MultiZone:Component:DetailedOpening"] = opening_array
                            


In [32]:
# Get the coords and CP values from idf
centroids = []
cps = []
for i, surface in enumerate(fenestration_objects):
    coordinates = [
            (surface.Vertex_1_Xcoordinate, surface.Vertex_1_Ycoordinate, surface.Vertex_1_Zcoordinate),
            (surface.Vertex_2_Xcoordinate, surface.Vertex_2_Ycoordinate, surface.Vertex_2_Zcoordinate),
            (surface.Vertex_3_Xcoordinate, surface.Vertex_3_Ycoordinate, surface.Vertex_3_Zcoordinate),
            (surface.Vertex_4_Xcoordinate, surface.Vertex_4_Ycoordinate, surface.Vertex_4_Zcoordinate)
        ]
    centroid = np.mean(coordinates, axis=0)
    cp = idf1.idfobjects['AirflowNetwork:MultiZone:WindPressureCoefficientValues'][i]['Wind_Pressure_Coefficient_Value_1']
    centroids.append(centroid)
    cps.append(cp)

# Plot the centroids and colour them based on the CP values
import plotly.graph_objects as go
# Create an empty np array with rows = len(fenestration_objects) and columns = 4 (x,y,z,CP)
import numpy as np
cp_array = np.empty((len(fenestration_objects), 4))
# Populate the cp_array with the centroids and CP values
for i, (centroid, cp) in enumerate(zip(centroids, cps)):
    cp_array[i] = np.array([*centroid, cp])


# Remove cp_array rows with CP values of -9999
cp_array = cp_array[cp_array[:,3] != -9999]

# Print min an max of CP values
print(f'Min CP value: {cp_array[:,3].min()}')
print(f'Max CP value: {cp_array[:,3].max()}')


# Scatter3D from cp_array and colour them based on the CP values
trace1 = go.Scatter3d(
    x=cp_array[:,0],
    y=cp_array[:,1],
    z=cp_array[:,2],
    mode='markers',
    marker=dict(size=2, color=cp_array[:,3], colorscale='Viridis', opacity=0.8)
)

fig = go.Figure(data=[trace1], layout=layout)
# Add a colorbar
fig.update_layout(coloraxis_colorbar=dict(
    title="CP Values",
    thicknessmode="pixels", thickness=50,
    lenmode="pixels", len=200,
    yanchor="top", y=1,
    ticks="outside", ticksuffix="",
    dtick=0.1
))

fig.show()

    

Min CP value: -2.7091204326893594
Max CP value: -2.02519149052436


In [10]:
idf1.newidfobject("Output:SQLite",
                  Option_Type = "Simple",
                  Unit_Conversion_for_Tabular_Data = "JtoKWH")
                  


OUTPUT:SQLITE,
    Simple,                   !- Option Type
    JtoKWH;                   !- Unit Conversion for Tabular Data

Here, we're checking each building surface to see if its boundary condition is set to 'Outdoors'. If so, we're adding the Cp values to it.

### Save Changes

Finally, let's save our modified IDF:

In [11]:
idf1.saveas('in_with_cp.idf')

This command will save the modified IDF as 'box_clean_with_cp.idf'. You can now run this new IDF in EnergyPlus to get the effects of the wind pressure coefficients.

Conclusion
You've now successfully added wind pressure coefficients to the surfaces in an E+ IDF file. This script can be adapted and expanded upon based on the specific requirements of your EnergyPlus models. Happy coding and modelling!